# HW5: Ensemble methods and cross validation with college data

Using the college statistic data to compare different ensemble methods on their performance with cross validation.


In [0]:
# import few things we will use in this exercise
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier

In [2]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.0MB/s 


In [0]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'185gv_hzqVvSUI_TBDINYNU6Leawu5tMo'}) # replace the id with id of file you want to access
downloaded.GetContentFile('college.csv')

In [0]:
# read in the data and save it in a dataframe
col = pd.read_csv('college.csv')

In [5]:
# Take a look and see what's in it
col.head(10)

,Unnamed: 0,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
5,Albertson College,Yes,587,479,158,38,62,678,41,13500,3335,500,675,67,73,9.4,11,9727,55
6,Albertus Magnus College,Yes,353,340,103,17,45,416,230,13290,5720,500,1500,90,93,11.5,26,8861,63
7,Albion College,Yes,1899,1720,489,37,68,1594,32,13868,4826,450,850,89,100,13.7,37,11487,73
8,Albright College,Yes,1038,839,227,30,63,973,306,15595,4400,300,500,79,84,11.3,23,11644,80
9,Alderson-Broaddus College,Yes,582,498,172,21,44,799,78,10468,3380,660,1800,40,41,11.5,15,8991,52


---
## EDA: 
##### You may perform more EDA steps than the ones suggested below.

In [6]:
# See what kind of data format and how many are there in each column
col.dtypes


Unnamed: 0      object
Private         object
Apps             int64
Accept           int64
Enroll           int64
Top10perc        int64
Top25perc        int64
F.Undergrad      int64
P.Undergrad      int64
Outstate         int64
Room.Board       int64
Books            int64
Personal         int64
PhD              int64
Terminal         int64
S.F.Ratio      float64
perc.alumni      int64
Expend           int64
Grad.Rate        int64
dtype: object

In [7]:
col.info()

# there are 777 in each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 19 columns):
Unnamed: 0     777 non-null object
Private        777 non-null object
Apps           777 non-null int64
Accept         777 non-null int64
Enroll         777 non-null int64
Top10perc      777 non-null int64
Top25perc      777 non-null int64
F.Undergrad    777 non-null int64
P.Undergrad    777 non-null int64
Outstate       777 non-null int64
Room.Board     777 non-null int64
Books          777 non-null int64
Personal       777 non-null int64
PhD            777 non-null int64
Terminal       777 non-null int64
S.F.Ratio      777 non-null float64
perc.alumni    777 non-null int64
Expend         777 non-null int64
Grad.Rate      777 non-null int64
dtypes: float64(1), int64(16), object(2)
memory usage: 115.4+ KB


In [8]:
col.describe()

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
count,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.00000
mean,3001.638353,2018.804376,779.972973,27.558559,55.796654,3699.907336,855.298584,10440.669241,4357.526384,549.380952,1340.642214,72.660232,79.702703,14.089704,22.743887,9660.171171,65.46332
std,3870.201484,2451.113971,929.176190,17.640364,19.804778,4850.420531,1522.431887,4023.016484,1096.696416,165.105360,677.071454,16.328155,14.722359,3.958349,12.391801,5221.768440,17.17771
min,81.000000,72.000000,35.000000,1.000000,9.000000,139.000000,1.000000,2340.000000,1780.000000,96.000000,250.000000,8.000000,24.000000,2.500000,0.000000,3186.000000,10.00000
25%,776.000000,604.000000,242.000000,15.000000,41.000000,992.000000,95.000000,7320.000000,3597.000000,470.000000,850.000000,62.000000,71.000000,11.500000,13.000000,6751.000000,53.00000
50%,1558.000000,1110.000000,434.000000,23.000000,54.000000,1707.000000,353.000000,9990.000000,4200.000000,500.000000,1200.000000,75.000000,82.000000,13.600000,21.000000,8377.000000,65.00000
75%,3624.000000,2424.000000,902.000000,35.000000,69.000000,4005.000000,967.000000,12925.000000,5050.000000,600.000000,1700.000000,85.000000,92.000000,16.500000,31.000000,10830.000000,78.00000
max,48094.000000,26330.000000,6392.000000,96.000000,100.000000,31643.000000,21836.000000,21700.000000,8124.000000,2340.000000,6800.000000,103.000000,100.000000,39.800000,64.000000,56233.000000,118.00000


---

#### Rename the first column name to University so it will be easier to relate.

In [9]:
#After renaming, check to see if the column names got changed.
col.rename(columns = {'Unnamed: 0': 'University'}, inplace = True)
col.head(2)


,University,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56


#### Change yes/no to 1/0 in column 'Priviate' to get it ready for model training.

In [10]:
#First of all, find out how many yes and no are there in the 'Private' column.

col['Private']. value_counts()

Yes    565
No     212
Name: Private, dtype: int64

In [15]:
#change yes to 1 and no to 0 in the "Private" column and view the changes after your action.
#col['Private'].loc[col['Private'] == 'Yes'] = 1
#col['Private'].loc[col['Private'] == 'No'] = 0
#col['Private'] = col['Private'].apply(lambda x: 1 if x == 'Yes' else 0)
col.Private = col.Private.apply(lambda x: 1 if x == 'Yes' else 0)

col.Private = LabelEncoder().fit_transform(col.Private)
col['Private']. head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: Private, dtype: int64

In [16]:
#check to see if there are equal number of 1/0s as that in yes/no you checked earilier.
col['Private']. value_counts()

# Yes, there are 565 'yes's and 212 'no's earilier, and now have the equal number of 1/0s with yes/nos

1    565
0    212
Name: Private, dtype: int64

___
### Compare classification performance of different models with cross validation

In [0]:
# Assign the Private column as target y and 
#assign all columns, except University and Private, in the dataframe as X
col.columns.values.tolist()

#X = col[['Apps','Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F.Undergrad', 'P.Undergrad', \
        # 'Outstate', 'Room.Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S.F.Ratio','perc.alumni', 'Expend', 'Grad.Rate']]
X = col.drop(['University', 'Private'], axis = 1)
y = col['Private']


#### Create a function that takes a model and the 'number of splits' parameter in StratifiedKFold cross validation

In [0]:
def cross_score(model,model_name,n_splits, X, y):
    """
    Function that shows performance avearge and standard deviation 
    with different models and differnt number of splits in cross validation.

    Args:
        model: machine learning model used, e.g., DecisionTreeClassifier()
        model_name: The name of the machine learning model used, e.g., 'Decision Tree' Classifier.
        n_splits: number of folds used in StratifiedKFold
        X: feature data used for the model
        y: target data used for the model
    Returns:
        none
    Printout: 
        the average and the standard deviation of cross validation score 
        with 3 decimal places like the following.
        
        Decision Tree Classifier with 2 splits has an average score of 0.916 ± 0.038

    """
    ### your code here ###
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True)
    s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
    print(s)
    print("{} Score:\t{:0.3} ± {:0.3}".format(model_name, s.mean().round(3), s.std().round(3)))

    

### Testing the function on Decision Tree algorithm

In [19]:
# show the  the average and the standard deviation of cross validation score 
#for splits ranging from 2 to 20

model = DecisionTreeClassifier()

for i in range(2,21):
  print("splits = {}".format(i))
  cross_score(model,"Decision Tree", i, X, y)


splits = 2
[0.92802057 0.87371134]
Decision Tree Score:	0.901 ± 0.027
splits = 3
[0.9        0.86486486 0.91085271]
Decision Tree Score:	0.892 ± 0.02
splits = 4
[0.91282051 0.90721649 0.92783505 0.90721649]
Decision Tree Score:	0.914 ± 0.008
splits = 5
[0.91025641 0.87179487 0.89677419 0.92258065 0.89032258]
Decision Tree Score:	0.898 ± 0.017
splits = 6
[0.92366412 0.94615385 0.85271318 0.91472868 0.89922481 0.86821705]
Decision Tree Score:	0.901 ± 0.032
splits = 7
[0.88392857 0.91071429 0.87387387 0.85585586 0.97297297 0.91818182
 0.94545455]
Decision Tree Score:	0.909 ± 0.038
splits = 8
[0.91836735 0.8877551  0.90816327 0.8877551  0.88659794 0.9375
 0.875      0.92708333]
Decision Tree Score:	0.904 ± 0.021
splits = 9
[0.94252874 0.90804598 0.94252874 0.93103448 0.86206897 0.89534884
 0.94186047 0.90588235 0.90588235]
Decision Tree Score:	0.915 ± 0.026
splits = 10
[0.91139241 0.89873418 0.92307692 0.97435897 0.88461538 0.88311688
 0.92207792 0.84415584 0.94805195 0.88311688]
Decision 

#### By examining the result above, which split(s) gives you best performance?

There is a bias-variance trade-oﬀ associated with the choice of k in k-fold cross-validation. Typically, given these considerations, one performs k-fold cross-validation using k = 5 or k = 10, as these values have been shown empirically to yield test error rate estimates that suﬀer neither from excessively high bias nor from very high variance. From the result above, we can see when k = 5 or k = 10, they have same mean and lower variance.


___
### Comparing the mean and sd of cross validation scores subject to decision tree and ensemble methods

Use the function you created above to print out  
the mean and sd of cross validation scores for  
decision trees, bagging, random forest, extra trees ,
AdaBoost, and Gradient Boosting classifiers   
with 3 folds cross validation.

In [0]:
n=3
model = DecisionTreeClassifier()
cross_score(model,"Decision Tree", n, X, y)


In [0]:
model = BaggingClassifier()
cross_score(model,"Bagging", n, X, y)

In [0]:
model1 = RandomForestClassifier()
cross_score(model1,"Random Forest", n, X, y)

In [0]:
model = ExtraTreesClassifier()
cross_score(model,"ExtraTrees", n, X, y)

In [0]:
model = AdaBoostClassifier()
cross_score(model,"AdaBoost", n, X, y)

In [0]:
model = GradientBoostingClassifier()
cross_score(model,"GradientBoosting", n, X, y)


#### Comment on the mean and the sd deviation of scores and see if they are consistent with the pros and cons of ensemble methods.   
You may want to run few times for generalized comparision purposes.

From the different methods, the mean is relatively stable, the std always change a lot after each run. 

1) Decision trees are sensitive to the specific data on which they are trained. If the training data is changed (e.g. a tree is trained on a subset of the training data) the resulting decision tree can be quite different and in turn the predictions can be quite different. **After running few times, the std sometimes very low,  or higher, or very high, each time was quite different, consistent with its cons.**

2) Bagging is the application of the Bootstrap procedure to a high-variance decision trees. When bagging with decision trees, we are less concerned about individual trees overfitting the training data. For this reason and for efficiency, the individual decision trees are grown deep (e.g. few training samples at each leaf-node of the tree) and the trees are not pruned. These trees will have both high variance and low bias. These are important characterize of sub-models when combining predictions using bagging.** So the mean of the score is quite good but the std is higher compare to the other algrithoms after a few times running.**

   1. Random forest changes the algorithm for the way that the sub-trees are learned so that the resulting predictions from all of the subtrees have less correlation. Combining predictions from multiple models in ensembles works better if the predictions from the sub-models are uncorrelated or at best weakly correlated. The random forest algorithm changes this procedure so that the learning algorithm is limited to a random sample of features of which to search. **After several times running, it shows the stable variance, the std doesn't change too much each time.**

   2. ExtraTrees classifier always tests random splits over fraction of features, in contrast to RandomForest, which tests all possible splits over fraction of features. ExtraTrees being a bit worse when there is a high number of noisy features. **After several times running,  the std change but not too much each time. From the comparation of current std of the three bagging methods, it is consistent with their pros and cons .**

3) Boosting is a an approach to calculate the output using several different models and then average the result using a weighted average approach. It focus new learners on examples that other get wrong, train learners sequentially and convert weak learners into strong classifier. The Boosting model has high accuracy when it works.

   1. AdaBoost algorithm uses to correct its predecessor is by paying more attention to under-fitted training instances by the previous model. But in noisy data when you have outliers, due to the nature of the algorithm it overfits on them. Therefore it is sensitive to outliers.** From the several running, the std is so different each time, shows the high variance. **

   2. Gradient Boosting works by sequentially adding the previous predictors underfitted predictions to the ensemble, ensuring the erros made previously are corrected. The difference lies in what it does with the underfitted values of its predecessor. Contrary to AdaBoost, which tweaks the instance weights at every interaction, this method tries to fit the new predictor to the residual errors made by the previous predictor.** From the ten times running, it shows the same mean and std occasionally, perhaps it means more accuracy.**


---

### Print out the "feature importances" of this best model.

The model has an attribute called `.feature_importances_` which can tell us which features were most important vs. others. It ranges from 0 to 1, with 1 being the most important.

An easy way to think about the feature importance is how much that particular variable was used to make decisions. Really though, it also takes into account how much that feature contributed to splitting up the class or reducing the variance.

A feature with higher feature importance reduced the criterion (impurity) more than the other features.

See 
<a href="https://towardsdatascience.com/running-random-forests-inspect-the-feature-importances-with-this-code-2b00dd72b92e">here </a> for more detail and coding sample.

#### Show, for RandomForestClassifier, the feature importances for each variable predicting private vs. not, sorted by most important feature to least.

In [0]:
# show the ranked list of feature importances here

model1.fit(X, y)

feature_importances = pd.DataFrame(model1.feature_importances_, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)

feature_importances


#### Show a ranked list of features in col dataframe that is correlated to the target column 'Private'

In [0]:
# ranked list of features in col dataframe that is correlated to the target column 'Private'
col.corr().abs().Private.sort_values(ascending = False)[1:]

#### Compare and comment on the feature importances and corr() lists you have above.


Provide your observation and interpretation here.

The most important features will be changed after each run since the random forest change the model every time, so it is not necessary to have some realationship with the correlation.

From the current results, we can see the features having higher correlation with the target variable, like F.Undergrad, Outstate, Enroll  have the more importance weight from the above cells.

So from the feature importance model, perhaps it tends to determine the features' correlation with the outcome variable. However, from the features' correlation can not determine the importance of the features.